In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy.stats import boxcox
from sklearn.metrics import mean_squared_error as mse
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
from sklearn import ensemble

In [2]:
# Import data
train = pd.read_csv('train.csv', index_col = 0)
test = pd.read_csv('test.csv', index_col = 0)

#get info

def get_details(data):
    skew= data.skew()
    corr = data.corr()['SalePrice']

    nulls = data.apply(lambda x: x.isnull().sum())
    nulls_perc = data.apply(lambda x: x.isnull().sum()/data.shape[0]*100)
    unique = data.apply(lambda x: [x.unique()])

    details = pd.concat([skew, corr, nulls, nulls_perc, unique], axis=1, sort=False)
    details.columns = ['skew', 'corr', 'nulls', 'nulls_perc', 'unique']
    
    return details

cat = train.select_dtypes(include=['O']).columns
cont = train.select_dtypes(exclude=['O']).columns

details = get_details(train[cont]).sort_values('corr', ascending = False).iloc[1:].head(12)

train = train.loc[train.GrLivArea <= 4500]
train = train.loc[train.TotalBsmtSF < 6000]
train = train.loc[train['1stFlrSF'] < 4000]

data = pd.concat([train, test], sort=False).reset_index(drop=True)

#impute NaN

data['PoolQC'].loc[data.PoolQC.isnull() & data.PoolArea == 0] = 'NA'
data['MiscFeature'].loc[data.MiscFeature.isnull() & (data.MiscVal == 0)] = 'NA'
data['Alley'].loc[data.Alley.isnull()] = 'NA'
data['Fence'].loc[data.Fence.isnull()] = 'NA'
data['FireplaceQu'].loc[data.FireplaceQu.isnull()] = 'NA'
data['LotFrontage'].loc[data.LotFrontage.isnull()] = 0

fill = pd.Series([data[c].value_counts().index[0] for c in data[cat]],
            index=data[cat].columns)

data[cat] = data[cat].fillna(fill)

details = get_details(data)

fill = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2','BsmtUnfSF', 'TotalBsmtSF', 'GarageArea']

for i in fill:
    data.loc[data[i].isnull(), i] = data.groupby(['OverallQual', 'Neighborhood'])[i].transform(lambda x: x.fillna(x.mean()))
    
fill = ['BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageYrBlt']

for i in fill:
    data.loc[data[i].isnull(), i] = data.groupby(['OverallQual', 'Neighborhood'])[i].transform(lambda x: x.fillna(x.median()))
    
data.loc[data['GarageYrBlt'].isnull(), 'GarageYrBlt'] = data.GarageYrBlt.median()
    
data = data.drop(['Utilities'], axis = 1)
    
cat = cat.to_list()
cat.remove('Utilities')

dummy = pd.get_dummies(data[cat])

data = pd.concat([data, dummy], axis =1)
data = data.drop(cat, axis = 1)

data.loc[data['SalePrice'].isnull(), 'SalePrice'] = 0

details = get_details(data[cont]).sort_values('skew', ascending = False)

no_log = data

cols = details.loc[abs(details['skew']) > 0.7].index.to_list()
cols.remove('SalePrice')

# Boxcox
x = data[cols]
l_opt = {}

for i in cols:
    x.loc[:, i], l_opt[i] = boxcox(x.loc[:, i] + 1)

data.loc[:, 'SalePrice'], l_opt = boxcox(data.loc[:, 'SalePrice'] + 1) 
    
data[cols] = x[cols]

data = data.drop('MiscVal', axis = 1)
data_copy = data

#go back to train and test
train  = data.loc[data.SalePrice > 0,]
test = data.loc[data.SalePrice == 0,]

x = train.drop('SalePrice', axis = 1)
y = train['SalePrice']

#pvalues

results = sm.OLS(y, x).fit()
pValues = results.pvalues

# pValues = list(pValues[pValues<0.05].index)
# x = x[pValues]

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2)

/home/sofiawangy/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/sofiawangy/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
Y_train

1049    11.951087
485     11.772173
54      11.651857
773     12.738900
554     11.514622
1301    12.218661
464     11.605589
305     12.489055
1451    11.229915
1201    11.548688
1059    12.137375
719     12.384943
1140    12.061167
827     11.997195
1340    12.203793
252     11.931570
1306    11.772173
258     12.216551
1012    11.235696
367     11.885235
546     11.648084
1341    11.953939
615     11.987628
386     11.188463
153     12.231227
1058    12.166712
53      12.713843
108     11.531804
434     11.188463
618     12.486175
          ...    
1128    11.724411
1167    12.231227
628     11.905778
761     12.145134
1277    12.239516
1076    11.715983
219     11.898432
682     12.419867
1275    11.867277
489     11.247156
347     11.839705
189     12.423292
767     12.152546
414     12.201650
461     11.824044
1156    12.210192
196     12.507957
310     11.888788
1225    12.139663
1319    11.079815
503     12.433494
1211    11.514622
1361    11.836169
1098    11.836593
939     11

In [3]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, Y_train)
print('train tree: ', tree_model.score(X_train, Y_train))
print('test tree: ', tree_model.score(X_test, Y_test))

ValueError: Unknown label type: 'continuous'